In [28]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical


df = pd.read_csv('modified.csv')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,2.128232,0.530628,0.00,1.064711,0.073250,2.484907,3.555348,0.9978,3.51,0.444686,2.341806,5
1,2.174752,0.631272,0.00,1.280934,0.093490,3.258097,4.219508,0.9968,3.20,0.518794,2.379546,5
2,2.174752,0.565314,0.04,1.193922,0.088011,2.772589,4.007333,0.9970,3.26,0.500775,2.379546,5
3,2.501436,0.246860,0.56,1.064711,0.072321,2.890372,4.110874,0.9980,3.16,0.457425,2.379546,6
4,2.128232,0.530628,0.00,1.064711,0.073250,2.484907,3.555348,0.9978,3.51,0.444686,2.341806,5


In [29]:
df.shape

(1599, 12)

In [30]:
df['quality'].value_counts()
def map_quality(val):
    if val in [3,4]:
        return 0
    elif val in [5,6]:
        return 1
    else:
        return 2

df['quality_class'] = df['quality'].apply(map_quality)

In [35]:
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality_class
0,2.128232,0.530628,0.00,1.064711,0.073250,2.484907,3.555348,0.99780,3.51,0.444686,2.341806,1
1,2.174752,0.631272,0.00,1.280934,0.093490,3.258097,4.219508,0.99680,3.20,0.518794,2.379546,1
2,2.174752,0.565314,0.04,1.193922,0.088011,2.772589,4.007333,0.99700,3.26,0.500775,2.379546,1
3,2.501436,0.246860,0.56,1.064711,0.072321,2.890372,4.110874,0.99800,3.16,0.457425,2.379546,1
4,2.128232,0.530628,0.00,1.064711,0.073250,2.484907,3.555348,0.99780,3.51,0.444686,2.341806,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,1.974081,0.470004,0.08,1.098612,0.086178,3.496508,3.806662,0.99490,3.45,0.457425,2.442347,1
1595,1.931521,0.438255,0.10,1.163151,0.060154,3.688879,3.951244,0.99512,3.52,0.565314,2.501436,1
1596,1.987874,0.412110,0.13,1.193922,0.073250,3.401197,3.713572,0.99574,3.42,0.559616,2.484907,1
1597,1.931521,0.497740,0.12,1.098612,0.072321,3.496508,3.806662,0.99547,3.57,0.536493,2.415914,1


In [32]:
df.drop(columns=['quality'], inplace=True)

In [33]:
X = df.drop('quality_class', axis=1)
y = df['quality_class']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
encoder = LabelEncoder()
y_train_enc = encoder.fit_transform(y_train)
y_test_enc = encoder.transform(y_test)

num_classes = len(encoder.classes_)  # number of unique classes
y_train_cat = to_categorical(y_train_enc, num_classes=num_classes)
y_test_cat = to_categorical(y_test_enc, num_classes=num_classes)

In [ ]:
def build_model():
    model = Sequential()
    model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(num_classes, activation='softmax')) 
    return model

model = build_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    X_train, y_train_cat,
    validation_split=0.2,
    epochs=100,
    batch_size=32
)

test_loss, test_acc = model.evaluate(X_test, y_test_cat, verbose=0)
print("✅ Test Accuracy:", test_acc)



Epoch 1/100


c:\Users\Manoj\anaconda4\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.7908 - loss: 0.6827 - val_accuracy: 0.7891 - val_loss: 0.6069
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8368 - loss: 0.5029 - val_accuracy: 0.8125 - val_loss: 0.5329
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8436 - loss: 0.4740 - val_accuracy: 0.8086 - val_loss: 0.5153
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8436 - loss: 0.4492 - val_accuracy: 0.8125 - val_loss: 0.5057
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8328 - loss: 0.4502 - val_accuracy: 0.8125 - val_loss: 0.4949
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8338 - loss: 0.4296 - val_accuracy: 0.8164 - val_loss: 0.4934
Epoch 7/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8358 - loss: 0.4187 - val_accuracy: 0.8164 - val_loss: 0.4809
Epoch 8/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8485 - loss: 0.4140 - val_accuracy: 0.8242 - val_loss: 0.